In [5]:
import warnings

warnings.filterwarnings('ignore')
import pandas as pd


from datetime import date
import psycopg2

from surprise import Dataset
from surprise import Reader

from surprise import SVD

def start_recommendation():
    con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cgzkthavydhk.us-east-2.redshift.amazonaws.com', 
    port= '5439', user= 'mantix', password= 'Mantix123!')
    cur = con.cursor()
    items_data = pd.read_sql("select * from cdp.items_data;",con)
    orders_data = pd.read_sql("select * from cdp.orders_data;",con)

    con.close()
    reader = Reader(rating_scale=(1, 5))

    locations = items_data.fulfillment_location_code.unique()
    orders_data.order_number = orders_data.order_number.astype(int).astype(str)
    items_data.order_number = items_data.order_number.astype(int).astype(str)
    ord_col = ['order_number','customer_account_id']
    item_col = ['order_number','fulfillment_location_code','product_code','quantity']
    #[customerid, productid, quantity]
    recommend_res = pd.DataFrame(columns=['customer_account_id','rec_prod'])
    print(f"Total Location : {len(locations)}")
    for loc in locations:
        items_filter = items_data[items_data.fulfillment_location_code==loc][item_col]
        df =pd.merge(orders_data[ord_col],items_filter,on=['order_number'],how='left')
        df = df[['customer_account_id','product_code','quantity']]
        df = df[~df.customer_account_id.isin(recommend_res.customer_account_id)]
        df = df.groupby(['customer_account_id','product_code'])['quantity'].sum().reset_index()
        df = df.dropna()

        filter_df =df.groupby(['product_code'])['quantity'].sum().reset_index().sort_values(by='quantity')
        index = filter_df.shape[0] - (filter_df.shape[0] * 0.05)
        req_prod_list = filter_df['product_code'].tolist()[round(index):]


        data = Dataset.load_from_df(df[['customer_account_id', 'product_code', 'quantity']], reader)

        # Train model
        svd = SVD(n_epochs=10)
        trainset = data.build_full_trainset()
        svd.fit(trainset)
        
        customer_list = df.customer_account_id.unique()
        prod_list = df.product_code.unique()
        prod_list = [prod for prod in prod_list if prod in req_prod_list]
        print(f'Location: {loc} |Total customer : {len(customer_list)} | Total Prod : {len(prod_list)}')
        for i,cust_id in enumerate(customer_list):
            d = pd.DataFrame( columns=['uid', 'iid', 'rui', 'est', 'details'])
            remove_prod_list = df[df.customer_account_id==cust_id]['product_code'].tolist()
            for prod in prod_list:
                if prod not in remove_prod_list:
                    d.loc[len(d)] = svd.predict(uid=cust_id, iid=prod)
            top_rec = d.sort_values(by='est',ascending=False)['iid'].tolist()[:10]
            recommend_res.loc[len(recommend_res)] = [cust_id, ','.join(top_rec)]
            #break

        
        #print(df.shape)
        #break

    today_date = date.today()

    recommend_res['create_dms'] = today_date
    recommend_res['tenant_id']='TNB00001'
    recommend_res['store_id']='store_id'
    recommend_res = recommend_res.rename(columns={'rec_prod':'product_code'})
    return recommend_res


def load_db(df):
    con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cgzkthavydhk.us-east-2.redshift.amazonaws.com', 
    port= '5439', user= 'mantix', password= 'Mantix123!')
    cur = con.cursor()

    def drop_table(conn,table):
        try:
            query = f"""DROP TABLE IF EXISTS {table}"""
            cur = conn.cursor()
            cur.execute(query)
            conn.commit()
            print(f'Drop table {table} successfully')
            #cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            #cur.close()
            #return 1

    def create_user_recommendation_table(conn,table):
        try:
            query = f"""CREATE TABLE {table} (customer_account_id VARCHAR(1000),
                            recommendation_product VARCHAR(5000),
                            tenant_id VARCHAR(1000),
                            store_id VARCHAR(1000),
                            create_dms VARCHAR(1000)
                        )"""
            cur = conn.cursor()
            cur.execute("ROLLBACK")
            conn.commit()
            cur.execute(query)
            conn.commit()
            print(f'Successfully created the table {table}')
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cur.close()
            return 1
            
    def single_insert(conn, cursor,insert_req):
        """ Execute a single INSERT request """
        #cursor = conn.cursor()
        try:
            cursor.execute(insert_req)
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            #cursor.close()
            return 1
        #cursor.close()
        
    def insert_dataframe(df,conn,cursor,table):
        for i in df.index:
            query = f"""
            INSERT into {table} (customer_account_id,recommendation_product ,tenant_id ,store_id,create_dms)
                values('%s',%s,%s,%s,%s);
            """ %(df.loc[i]['customer_account_id'],
                "'"+str(df.loc[i]['product_code'])+"'",
                    "'"+str(df.loc[i]['tenant_id'])+"'",
                "'"+str(df.loc[i]['store_id'])+"'",
                "'"+str(df.loc[i]['create_dms'])+"'")
            if i%50==0:
                print(i,end=' ')
            
            # check if customer_id and create_dms is already exist then don't insert
            check_query = f"SELECT * FROM {table} WHERE customer_account_id='{df.loc[i]['customer_account_id']}' and create_dms='{df.loc[i]['create_dms']}'"
            check_insert_df = pd.read_sql(check_query,conn)
            if len(check_insert_df) == 0:
                single_insert(conn,cursor, query)
            else:
                print('Customer already exist')
        conn.commit()
        print('\nSuccessfully inserted dataframe into the table')
        
    table = 'cdp.user_recommendations'
    drop_table(con,table)
    create_user_recommendation_table(con,table)
    insert_dataframe(df,con,cur,table)

if __name__ == '__main__':

    # start customer recommendation
    print('~ Start recommendation')
    recommend_res = start_recommendation()
    # load data into database
    print('~ Start to insert recommendation result into database')
    load_db(recommend_res)



ModuleNotFoundError: No module named 'surprise'

In [7]:
!pip install scikit-surprise

  Using cached scikit-surprise-1.1.1.tar.gz (11.8 MB)
  Running setup.py clean for scikit-surprise
Failed to build scikit-surprise
    Running setup.py install for scikit-surprise: started
    Running setup.py install for scikit-surprise: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\win10\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\win10\\AppData\\Local\\Temp\\pip-install-1j2rjtlj\\scikit-surprise\\setup.py'"'"'; __file__='"'"'C:\\Users\\win10\\AppData\\Local\\Temp\\pip-install-1j2rjtlj\\scikit-surprise\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\win10\AppData\Local\Temp\pip-wheel-lg4wp484'
       cwd: C:\Users\win10\AppData\Local\Temp\pip-install-1j2rjtlj\scikit-surprise\
  Complete output (49 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\surprise
  copying surprise\accuracy.py -> build\lib.win-amd64-3.8\surprise
  copying surprise\builtin_datasets.py -> build\lib.win-amd64-3.8\surprise
